In [3]:
import sys

sys.path.append("..")  # Add the parent directory of 'examples' to the Python path

## Addresses

Create an `Address` object from a _bech32-encoded_ string:

In [4]:
from dharitri_py_sdk import Address

address = Address.new_from_bech32("drt1c7pyyq2yaq5k7atn9z6qn5qkxwlc6zwc4vg7uuxn9ssy7evfh5jq4nm79l")

print("Address (bech32-encoded)", address.to_bech32())
print("Public key (hex-encoded):", address.to_hex())
print("Public key (hex-encoded):", address.get_public_key().hex())

ModuleNotFoundError: No module named 'Cryptodome'

... or from a _hex-encoded_ string - note that you have to provide the address prefix, also known as the **HRP** (_human-readable part_ of the address):

In [ ]:
address = Address.new_from_hex("c782420144e8296f757328b409d01633bf8d09d8ab11ee70d32c204f6589bd24", "drt")

... or from a raw public key:

In [ ]:
pubkey = bytes.fromhex("c782420144e8296f757328b409d01633bf8d09d8ab11ee70d32c204f6589bd24")
address = Address(pubkey, "drt")

Alternatively, you can use an `AddressFactory` (initialized with a specific **HRP**) to create addresses:

In [ ]:
from dharitri_py_sdk import AddressFactory

factory = AddressFactory("drt")

address = factory.create_from_bech32("drt1c7pyyq2yaq5k7atn9z6qn5qkxwlc6zwc4vg7uuxn9ssy7evfh5jq4nm79l")
address = factory.create_from_hex("c782420144e8296f757328b409d01633bf8d09d8ab11ee70d32c204f6589bd24")
address = factory.create_from_public_key(bytes.fromhex("c782420144e8296f757328b409d01633bf8d09d8ab11ee70d32c204f6589bd24"))

Addresses can be converted from one representation to another as follows:

In [ ]:
print(address.to_bech32())
print(address.to_hex())

Getting the shard of an address:

In [ ]:
from dharitri_py_sdk import AddressComputer

address_computer = AddressComputer(number_of_shards=3)
print("Shard:", address_computer.get_shard_of_address(address))

Checking whether an address is a smart contract:

In [ ]:
address = Address.new_from_bech32("drt1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qwgcss9")

print("Is contract:", address.is_smart_contract())

## REWA / DCDT transfers

Create an REWA transfer:

In [ ]:
from dharitri_py_sdk import Transaction, TransactionsConverter

transaction = Transaction(
    sender="drt1c7pyyq2yaq5k7atn9z6qn5qkxwlc6zwc4vg7uuxn9ssy7evfh5jq4nm79l",
    receiver="drt18h03w0y7qtqwtra3u4f0gu7e3kn2fslj83lqxny39m5c4rwaectswerhd2",
    gas_limit=50000,
    chain_id="D",
    nonce=77,
    value=1000000000000000000
)

transaction_converter = TransactionsConverter()
print(transaction_converter.transaction_to_dictionary(transaction))

In case you are using a **guarded** account you should also populate the `guardian` and `guardian_signature` fields after creating the transaction.

We'll see later how to [sign](#signing-objects) and [broadcast](#broadcasting-transactions) a transaction.

Create an REWA transfer, but this time with a payload (data):

In [ ]:
transaction = Transaction(
    sender="drt1c7pyyq2yaq5k7atn9z6qn5qkxwlc6zwc4vg7uuxn9ssy7evfh5jq4nm79l",
    receiver="drt18h03w0y7qtqwtra3u4f0gu7e3kn2fslj83lqxny39m5c4rwaectswerhd2",
    gas_limit=50000,
    chain_id="D",
    nonce=77,
    value=1000000000000000000,
    data=b"for the book"
)

print(transaction_converter.transaction_to_dictionary(transaction))

Alternatively, we can create an REWA transfer using a **transaction factory** (as we will see below, transaction factories are more commonly used). But before that, we have to create a configuration object (for any factory that we might use):

In [ ]:
from dharitri_py_sdk import TransactionsFactoryConfig

config = TransactionsFactoryConfig(chain_id="D")

The **transaction factory** is parametrized at instantiation, and the transaction is obtained by invoking the `create_transaction...` method:

In [ ]:
from dharitri_py_sdk import TransferTransactionsFactory

transfer_factory = TransferTransactionsFactory(config=config)
alice = Address.new_from_bech32("drt1c7pyyq2yaq5k7atn9z6qn5qkxwlc6zwc4vg7uuxn9ssy7evfh5jq4nm79l")
bob = Address.new_from_bech32("drt18h03w0y7qtqwtra3u4f0gu7e3kn2fslj83lqxny39m5c4rwaectswerhd2")

# With "data" field
transaction = transfer_factory.create_transaction_for_native_token_transfer(
    sender=alice,
    receiver=bob,
    native_amount=1000000000000000000,
    data="for the book"
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Create a single DCDT transfer:

In [ ]:
from dharitri_py_sdk import Token, TokenTransfer

token = Token("TEST-8b028f")
transfer = TokenTransfer(token, 10000)

transaction = transfer_factory.create_transaction_for_dcdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Create a single NFT transfer:

Keep in mind, since we are sending a NFT, we **should** set the amount to `1`.

In [ ]:
token = Token(identifier="TEST-38f249", nonce=1)
transfer = TokenTransfer(token=token, amount=1)

transaction = transfer_factory.create_transaction_for_dcdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Create a single SFT transfer (almost the same as above, the only difference being that for the transfer we set the desired amount, as an integer):

In [ ]:
token = Token(identifier="SEMI-9efd0f", nonce=1)
transfer = TokenTransfer(token=token, amount=5)

transaction = transfer_factory.create_transaction_for_dcdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Create a multiple DCDT / NFT transfer:

In [ ]:
first_token = Token(identifier="TEST-38f249", nonce=1)
first_transfer = TokenTransfer(token=first_token, amount=1)

second_token = Token(identifier="BAR-c80d29")
second_transfer = TokenTransfer(token=second_token, amount=10000000000000000000)

transaction = transfer_factory.create_transaction_for_dcdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[first_transfer, second_transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Additionally, we also have a method that combines the above methods and is able to identify the kind of transfer that we intend to perform based on it's parameters. The method can be used as follows:

For native token tranfers:

In [ ]:
from dharitri_py_sdk import TransferTransactionsFactory, TransactionsFactoryConfig

transfer_factory = TransferTransactionsFactory(config=TransactionsFactoryConfig(chain_id="D"))

alice = Address.new_from_bech32("drt1c7pyyq2yaq5k7atn9z6qn5qkxwlc6zwc4vg7uuxn9ssy7evfh5jq4nm79l")
bob = Address.new_from_bech32("drt18h03w0y7qtqwtra3u4f0gu7e3kn2fslj83lqxny39m5c4rwaectswerhd2")

# With "data" field
transaction = transfer_factory.create_transaction_for_transfer(
    sender=alice,
    receiver=bob,
    native_amount=1000000000000000000,
    data="for the book".encode()
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

And for DCDT transfers:

In [ ]:
first_token = Token(identifier="TEST-38f249", nonce=1)
first_transfer = TokenTransfer(token=first_token, amount=1)

second_token = Token(identifier="BAR-c80d29")
second_transfer = TokenTransfer(token=second_token, amount=10000000000000000000)

transaction = transfer_factory.create_transaction_for_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[first_transfer, second_transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Alternatively, the protocol will support (in the very near future) sending both native and dcdt tokens in the same transaction. If a `native_amount` is provided together with `token_transfers`, the native token will also be included in the `MultiDCDTNFTTrasfer` built-in function call.

In [ ]:
first_token = Token(identifier="TEST-38f249", nonce=1)
first_transfer = TokenTransfer(token=first_token, amount=1)

second_token = Token(identifier="BAR-c80d29")
second_transfer = TokenTransfer(token=second_token, amount=10000000000000000000)

transaction = transfer_factory.create_transaction_for_transfer(
    sender=alice,
    receiver=bob,
    native_amount=1000000000000000000,
    token_transfers=[first_transfer, second_transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

### Decoding Transactions

For example, when sending multiple DCDT and NFT tokens, the receiver field of the transaction is the same as the sender field and also the value is set to `0` because all the information is encoded in the `data` field of the transaction.

For decoding the data field we have a so called `TransactionDecoder`. We fetch the transaction from the network and then use the decoder.

In [ ]:
from dharitri_py_sdk import ProxyNetworkProvider, TransactionDecoder

proxy = ProxyNetworkProvider("https://devnet-api.dharitri.org")
transaction = proxy.get_transaction("3e7b39f33f37716186b6ffa8761d066f2139bff65a1075864f612ca05c05c05d")

decoder = TransactionDecoder()
decoded_transaction = decoder.get_transaction_metadata(transaction)

print(decoded_transaction.to_dict())

## Relayed Transactions

First, we get the newtwork configuration using the network providers.

In [ ]:
from dharitri_py_sdk import ProxyNetworkProvider

provider = ProxyNetworkProvider("https://devnet-gateway.dharitri.org")
network_config = provider.get_network_config()

### Relayed V1

This is still supported, but we advise you to use Relayed V3 instead.

In [ ]:
from pathlib import Path

from dharitri_py_sdk import (Address, RelayedTransactionsFactory, Transaction,
                            TransactionComputer, TransactionsFactoryConfig,
                            UserSigner)

signer = UserSigner.from_pem_file(Path("../dharitri_py_sdk/testutils/testwallets/bob.pem"))
transaction_computer = TransactionComputer()

inner_tx = Transaction(
    chain_id=network_config.chain_id,
    sender="drt18h03w0y7qtqwtra3u4f0gu7e3kn2fslj83lqxny39m5c4rwaectswerhd2",
    receiver="drt1qqqqqqqqqqqqqpgqqczn0ccd2gh8eqdswln7w9vzctv0dwq7d8ssxfnjku",
    gas_limit=60000000,
    nonce=198,
    data=b"add@05"
)
inner_tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(inner_tx))

config = TransactionsFactoryConfig(chain_id="D")
factory = RelayedTransactionsFactory(config=config)
relayer = Address.new_from_bech32("drt1c7pyyq2yaq5k7atn9z6qn5qkxwlc6zwc4vg7uuxn9ssy7evfh5jq4nm79l")

relayed_tx = factory.create_relayed_v1_transaction(
    inner_transaction=inner_tx,
    relayer_address=relayer
)
relayed_tx.nonce = 2627

print(transaction_converter.transaction_to_dictionary(relayed_tx))

### Relayed V2

This is still supported, but we advise you to use Relayed V3 instead.

In [ ]:
from pathlib import Path

from dharitri_py_sdk import (Address, RelayedTransactionsFactory, Transaction,
                            TransactionComputer, TransactionsFactoryConfig,
                            UserSigner)

signer = UserSigner.from_pem_file(Path("../dharitri_py_sdk/testutils/testwallets/bob.pem"))
transaction_computer = TransactionComputer()

# for the relayedV2 transactions, the gasLimit for the inner transaction should be 0
inner_tx = Transaction(
            chain_id=network_config.chain_id,
            sender="drt18h03w0y7qtqwtra3u4f0gu7e3kn2fslj83lqxny39m5c4rwaectswerhd2",
            receiver="drt1qqqqqqqqqqqqqpgqqczn0ccd2gh8eqdswln7w9vzctv0dwq7d8ssxfnjku",
            gas_limit=0,
            nonce=15,
            data=b"add@05"
        )
inner_tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(inner_tx))

config = TransactionsFactoryConfig(chain_id="D")
factory = RelayedTransactionsFactory(config=config)
relayer = Address.new_from_bech32("drt1c7pyyq2yaq5k7atn9z6qn5qkxwlc6zwc4vg7uuxn9ssy7evfh5jq4nm79l")

relayed_tx = factory.create_relayed_v2_transaction(
    inner_transaction=inner_tx,
    inner_transaction_gas_limit=60_000_000,
    relayer_address=relayer
)
relayed_tx.nonce = 37

print(transaction_converter.transaction_to_dictionary(relayed_tx))

### Relayed V3

This is the latest iteration of relayed transactions and we highly recommend using this version over the previous ones as soon as it becomes available on the network.

In [ ]:
from pathlib import Path

from dharitri_py_sdk import (Address, RelayedTransactionsFactory, Transaction,
                            TransactionComputer, TransactionsFactoryConfig,
                            UserSigner)

bob = UserSigner.from_pem_file(Path("../dharitri_py_sdk/testutils/testwallets/bob.pem"))
relayer = Address.new_from_bech32("drt1c7pyyq2yaq5k7atn9z6qn5qkxwlc6zwc4vg7uuxn9ssy7evfh5jq4nm79l")

inner_tx = Transaction(
            chain_id=network_config.chain_id,
            sender="drt18h03w0y7qtqwtra3u4f0gu7e3kn2fslj83lqxny39m5c4rwaectswerhd2",
            receiver="drt1qqqqqqqqqqqqqpgqqczn0ccd2gh8eqdswln7w9vzctv0dwq7d8ssxfnjku",
            gas_limit=60000,
            nonce=15,
            data=b"hello",
            relayer=relayer.to_bech32()
        )
transaction_computer = TransactionComputer()
inner_tx.signature = bob.sign(transaction_computer.compute_bytes_for_signing(inner_tx))

config = TransactionsFactoryConfig(chain_id="D")
factory = RelayedTransactionsFactory(config=config)

relayed_tx = factory.create_relayed_v3_transaction(
    relayer_address=relayer,
    inner_transactions=[inner_tx]
)

print(transaction_converter.transaction_to_dictionary(relayed_tx))

A few things worth mentioning before moving further, the `data` field of the relayer's transaction **should not** be set and the relayer should be in the same shard as the sender of the inner transaction.

Don't forget, after the Relayed V3 transaction is created, the `nonce` needs to be set and the transaction should be signed before broadcasting it.

## Contract ABIs

A contract's ABI describes the endpoints, data structure and events that a contract exposes. While contract interactions are possible without the ABI, they are easier to implement when the definitions are available.

### Load the ABI from a file

In [ ]:
from dharitri_py_sdk.abi import Abi, AbiDefinition

abi_definition = AbiDefinition.load(Path("./contracts/adder.abi.json"))
abi = Abi(abi_definition)

Or even simpler:

In [ ]:
abi = Abi.load(Path("./contracts/adder.abi.json"))

### Manually construct the ABI

If an ABI file isn't directly available, but you do have knowledge of the contract's endpoints and types, you can manually construct the ABI. Let's see a simple example:

In [ ]:
abi_definition = AbiDefinition.from_dict({
    "endpoints": [{
        "name": "add",
        "inputs": [
            {
                "name": "value",
                "type": "BigUint"
            }
        ],
        "outputs": []
    }]
})

An endpoint with both inputs and outputs:

In [ ]:
abi_definition = AbiDefinition.from_dict({
    "endpoints": [
        {
            "name": "foo",
            "inputs": [
                { "type": "BigUint" },
                { "type": "u32" },
                { "type": "Address" }
            ],
            "outputs": [
                { "type": "u32" }
            ]
        },
        {
            "name": "bar",
            "inputs": [
                { "type": "counted-variadic<utf-8 string>" },
                { "type": "variadic<u64>" }
            ],
            "outputs": []
        }
    ]
})

## Contract deployments

### Load the bytecode from a file

In [ ]:
from pathlib import Path

bytecode = Path("contracts/adder.wasm").read_bytes()

### Perform a contract deployment

First, let's create a `SmartContractTransactionsFactory`:

In [ ]:
from dharitri_py_sdk import SmartContractTransactionsFactory

factory = SmartContractTransactionsFactory(config)

If the contract ABI is available, provide it to the factory:

In [ ]:
abi = Abi.load(Path("contracts/adder.abi.json"))
factory = SmartContractTransactionsFactory(config, abi)

Now, prepare the deploy transaction:

In [ ]:
from dharitri_py_sdk.abi import U32Value

# For deploy arguments, use typed value objects if you haven't provided an ABI to the factory:
args = [U32Value(42)]
# Or use simple, plain Python values and objects if you have provided an ABI to the factory:
args = [42]

deploy_transaction = factory.create_transaction_for_deploy(
    sender=alice,
    bytecode=bytecode,
    arguments=args,
    gas_limit=10000000,
    is_upgradeable=True,
    is_readable=True,
    is_payable=True,
    is_payable_by_sc=True
)

print("Transaction:", transaction_converter.transaction_to_dictionary(deploy_transaction))
print("Transaction data:", deploy_transaction.data.decode())

When creating transactions using `SmartContractTransactionsFactory`, even if the ABI is available and provided,
you can still use _typed value_ objects as arguments for deployments and interactions.

Even further, you can use a mix of _typed value_ objects and plain Python values and objects. For example:
```
args = [U32Value(42), "hello", { "foo": "bar" }, TokenIdentifierValue("TEST-abcdef")];
```

Setting the transaction nonce, signing a transaction and broadcasting it are depicted in a later section.

### Computing the contract address

Even before broadcasting, at the moment you know the sender address and the nonce for your deployment transaction, you can (deterministically) compute the (upcoming) address of the contract:

In [ ]:
from dharitri_py_sdk import AddressComputer

address_computer = AddressComputer()
contract_address = address_computer.compute_contract_address(
    deployer=Address.new_from_bech32(deploy_transaction.sender),
    deployment_nonce=deploy_transaction.nonce
)

print("Contract address:", contract_address.to_bech32())

### Parsing transaction outcome

In the end, you can parse the results using a `SmartContractTransactionsOutcomeParser`. However, since the `parse_deploy` method requires a `TransactionOutcome` object as input, we need to first convert our `TransactionOnNetwork` object to a `TransactionOutcome`, by means of a `TransactionsConverter`.

In [ ]:
from dharitri_py_sdk import (SmartContractTransactionsOutcomeParser,
                            TransactionsConverter)

converter = TransactionsConverter()
parser = SmartContractTransactionsOutcomeParser()

transaction_on_network = proxy.get_transaction("0a7da74038244790b5bd4cd614c26cd5a6be76a6fcfcfb037974cc116b2ee9c6")
transaction_outcome = converter.transaction_on_network_to_outcome(transaction_on_network)
parsed_outcome = parser.parse_deploy(transaction_outcome)

print(parsed_outcome)

## Contract upgrades

Contract upgrade transactions are similar to deployment transactions (see above), in the sense that they also require a contract bytecode. In this context, though, the contract address is already known.

In [ ]:
contract_address = Address.new_from_bech32("drt1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qwgcss9")
bytecode = Path("./contracts/adder.wasm").read_bytes()

upgrade_transaction = factory.create_transaction_for_upgrade(
    sender=alice,
    contract=contract_address,
    bytecode=bytecode,
    gas_limit=10000000,
    arguments=[42],
    is_upgradeable=True,
    is_readable=True,
    is_payable=True,
    is_payable_by_sc=True
)

print("Transaction:", transaction_converter.transaction_to_dictionary(upgrade_transaction))
print("Transaction data:", upgrade_transaction.data.decode())

## Contract interactions

The recommended way to create transactions for calling (and, for that matter, deploying and upgrading) smart contracts is through a `SmartContractTransactionsFactory`.

In [ ]:
from dharitri_py_sdk import SmartContractTransactionsFactory

factory = SmartContractTransactionsFactory(config)

If the contract ABI is available, provide it to the factory:

In [ ]:
abi = Abi.load(Path("contracts/adder.abi.json"))
factory = SmartContractTransactionsFactory(config, abi)

### Regular interactions

Now, let's prepare a contract transaction, to call the `add` function of our previously deployed smart contract:

In [ ]:
contract_address = Address.new_from_bech32("drt1qqqqqqqqqqqqqpgqws44xjx2t056nn79fn29q0rjwfrd3m43396qzdr496")

# For arguments, use typed value objects if you haven't provided an ABI to the factory:
args = [U32Value(42)]
# Or use simple, plain Python values and objects if you have provided an ABI to the factory:
args = [42]

transaction = factory.create_transaction_for_execute(
    sender=alice,
    contract=contract_address,
    function="add",
    gas_limit=10000000,
    arguments=args
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

When creating transactions using `SmartContractTransactionsFactory`, even if the ABI is available and provided,
you can still use _typed value_ objects as arguments for deployments and interactions.

Even further, you can use a mix of _typed value_ objects and plain Python values and objects. For example:
```
args = [U32Value(42), "hello", { "foo": "bar" }, TokenIdentifierValue("TEST-abcdef")];
```

Setting the transaction nonce, signing a transaction and broadcasting it are depicted in a later section.

### Transfer & execute

At times, you may want to send some tokens (native REWA or DCDT) along with the contract call.

For transfer & execute with native REWA, prepare your transaction as follows:

In [ ]:
transaction = factory.create_transaction_for_execute(
    sender=alice,
    contract=contract_address,
    function="add",
    gas_limit=10000000,
    arguments=[42],
    native_transfer_amount=1000000000000000000
)

Above, we're sending 1 REWA along with the contract call.

For transfer & execute with DCDT tokens, prepare your transaction as follows:

In [ ]:
first_token = Token("TEST-38f249", 1)
first_transfer = TokenTransfer(first_token, 1)

second_token = Token("BAR-c80d29")
second_transfer = TokenTransfer(second_token, 10000000000000000000)

transfers = [first_transfer, second_transfer]

transaction = factory.create_transaction_for_execute(
    sender=alice,
    contract=contract_address,
    function="add",
    gas_limit=10000000,
    arguments=[42],
    token_transfers=transfers
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

### Parsing transaction outcome

Documentation in this section is preliminary and subject to change.

### Decode transaction events

You might be interested into decoding events emitted by a contract. You can do so by using the [`TransactionEventsParser`](#).

Suppose we'd like to decode a `startPerformAction` event emitted by the [multisig](https://github.com/TerraDharitri/drt-rs-contracts/tree/main/contracts/multisig) contract.

Let's fetch an already processed [transaction](https://devnet-explorer.dharitri.org/transactions/05d445cdd145ecb20374844dcc67f0b1e370b9aa28a47492402bc1a150c2bab4), to serve as an exmple, and convert it to a [TransactionOutcome](https://TerraDharitri.github.io/drt-py-sdk/_modules/dharitri_py_sdk/core/transactions_outcome_parsers/resources.html#TransactionOutcome).

In [ ]:
from dharitri_py_sdk import ApiNetworkProvider, TransactionsConverter

api = ApiNetworkProvider("https://testnet-api.dharitri.org")
converter = TransactionsConverter()

transaction_on_network = api.get_transaction("6f006c99e45525c94629db2442d9ca27ff088ad113a09f0a3a3e24bcc164945a")
transaction_outcome = converter.transaction_on_network_to_outcome(transaction_on_network)

Now, lets find and parse the event we are interested in.

In [ ]:
from dharitri_py_sdk import TransactionEventsParser, find_events_by_first_topic
from dharitri_py_sdk.abi import Abi

abi = Abi.load(Path("./contracts/multisig-full.abi.json"))
events_parser = TransactionEventsParser(abi)

[event] = find_events_by_first_topic(transaction_outcome, "startPerformAction")
parsed_event = events_parser.parse_event(event)

print(parsed_event)

## Contract queries

In order to perform Smart Contract queries, we recommend the use of `SmartContractQueriesController`.

You will notice that the `SmartContractQueriesController` requires a `QueryRunner` object at initialization. A `NetworkProvider`, slighly adapted, is used to satisfy this requirement (more details about **network providers** can be found in a later section).

In [ ]:
from dharitri_py_sdk import (ProxyNetworkProvider, QueryRunnerAdapter,
                            SmartContractQueriesController)

contract = Address.new_from_bech32("drt1qqqqqqqqqqqqqpgqqy34h7he2ya6qcagqre7ur7cc65vt0mxrc8qwq64qt")
query_runner = QueryRunnerAdapter(ProxyNetworkProvider("https://devnet-api.dharitri.org"))

query_controller = SmartContractQueriesController(query_runner)


If the contract ABI is available, provide it to the controller:

In [ ]:
abi = Abi.load(Path("contracts/adder.abi.json"))
query_controller = SmartContractQueriesController(query_runner, abi)

Query the contract as follows:

In [ ]:
data_parts = query_controller.query(
    contract=contract.to_bech32(),
    function="getSum",
    arguments=[],
)

print("Return data (parsed):", data_parts)

For finer control, first create a contract query, then run it and parse the outcome at a later time:

In [ ]:
query = query_controller.create_query(
    contract=contract.to_bech32(),
    function="getSum",
    arguments=[],
)

response = query_controller.run_query(query)
data_parts = query_controller.parse_query_response(response)

print("Return code:", response.return_code)
print("Return data (raw):", response.return_data_parts)
print("Return data (parsed):", data_parts)

## Creating wallets

Mnemonic generation is based on [`trezor/python-mnemonic`](https://github.com/trezor/python-mnemonic) and can be achieved as follows:

In [ ]:
from dharitri_py_sdk import Mnemonic

mnemonic = Mnemonic.generate()
words = mnemonic.get_words()

print(words)

The mnemonic can be saved to a keystore file:

In [ ]:
from pathlib import Path
from dharitri_py_sdk import UserWallet

path = Path("./output")
if not path.exists():
    path.mkdir(parents=True, exist_ok=True)

wallet = UserWallet.from_mnemonic(mnemonic.get_text(), "password")
wallet.save(path / "walletWithMnemonic.json")

Given a mnemonic, one can derive keypairs:

In [ ]:
secret_key = mnemonic.derive_key(0)
public_key = secret_key.generate_public_key()

print("Secret key:", secret_key.hex())
print("Public key:", public_key.hex())

A keypair can be saved as a JSON wallet:

In [ ]:
path = Path("./output")
if not path.exists():
    path.mkdir(parents=True, exist_ok=True)

wallet = UserWallet.from_secret_key(secret_key, "password")
wallet.save(path / "wallet.json", address_hrp="drt")

... or as a PEM wallet (usually not recommended):

In [ ]:
from dharitri_py_sdk import Address, UserPEM

path = Path("./output")
if not path.exists():
    path.mkdir(parents=True, exist_ok=True)

label = Address(public_key.buffer, "drt").to_bech32()
pem = UserPEM(label=label, secret_key=secret_key)
pem.save(path / "wallet.pem")

## Loading wallets

This is not a very common use-case - you might refer to [signing objects](#signing-objects) instead.

Load a keystore that holds an **encrypted mnemonic** (and perform wallet derivation at the same time):

In [ ]:
from dharitri_py_sdk import UserWallet

secret_key = UserWallet.load_secret_key(Path("../dharitri_py_sdk/testutils/testwallets/withDummyMnemonic.json"), "password", address_index=0)
address = secret_key.generate_public_key().to_address("drt")

print("Secret key:", secret_key.hex())
print("Address:", address.to_bech32())

Load a keystore that holds an **encrypted secret** key:

In [ ]:
secret_key = UserWallet.load_secret_key(Path("../dharitri_py_sdk/testutils/testwallets/alice.json"), "password")
address = secret_key.generate_public_key().to_address("drt")

print("Secret key:", secret_key.hex())
print("Address:", address.to_bech32())

Load the secret key from a PEM file:

In [ ]:
from dharitri_py_sdk import UserPEM

pem = UserPEM.from_file(Path("../dharitri_py_sdk/testutils/testwallets/alice.pem"))

print("Secret key:", pem.secret_key.hex())
print("Public key:", pem.public_key.hex())

## Signing objects

Creating a `UserSigner` from a JSON wallet:

In [ ]:
from dharitri_py_sdk import UserSigner

signer = UserSigner.from_wallet(Path("../dharitri_py_sdk/testutils/testwallets/alice.json"), "password")

Creating a `UserSigner` from a PEM file:

In [ ]:
signer = UserSigner.from_pem_file(Path("../dharitri_py_sdk/testutils/testwallets/alice.pem"))

Signing a transaction:

In [ ]:
from dharitri_py_sdk import Transaction, TransactionComputer

tx = Transaction(
    nonce=90,
    sender="drt1c7pyyq2yaq5k7atn9z6qn5qkxwlc6zwc4vg7uuxn9ssy7evfh5jq4nm79l",
    receiver="drt18h03w0y7qtqwtra3u4f0gu7e3kn2fslj83lqxny39m5c4rwaectswerhd2",
    value=1000000000000000000,
    gas_limit=50000,
    chain_id="D"
)

transaction_computer = TransactionComputer()

tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx))
print("Signature:", tx.signature.hex())

Signing an arbitrary message:

In [ ]:
from dharitri_py_sdk import Message, MessageComputer

signer_address = signer.get_pubkey().to_address(hrp="drt")
message = Message(b"hello")
message_computer = MessageComputer()

message.signature = signer.sign(message_computer.compute_bytes_for_signing(message))

print("Signature:", message.signature.hex())

## Verifying signatures

Creating a `UserVerifier`:

In [ ]:
from dharitri_py_sdk import Address, UserVerifier

alice = Address.new_from_bech32("drt1c7pyyq2yaq5k7atn9z6qn5qkxwlc6zwc4vg7uuxn9ssy7evfh5jq4nm79l")
bob = Address.new_from_bech32("drt18h03w0y7qtqwtra3u4f0gu7e3kn2fslj83lqxny39m5c4rwaectswerhd2")

alice_verifier = UserVerifier.from_address(alice)
bob_verifier = UserVerifier.from_address(bob)

Verifying a signature:

In [ ]:
from dharitri_py_sdk import MessageComputer, TransactionComputer

transaction_computer = TransactionComputer()
message_computer = MessageComputer()

print(f"Is signature of Alice?", alice_verifier.verify(transaction_computer.compute_bytes_for_signing(tx), tx.signature))
print(f"Is signature of Alice?", alice_verifier.verify(message_computer.compute_bytes_for_verifying(message), message.signature))
print(f"Is signature of Bob?", bob_verifier.verify(transaction_computer.compute_bytes_for_signing(tx), tx.signature))
print(f"Is signature of Bob?", bob_verifier.verify(message_computer.compute_bytes_for_verifying(message), message.signature))

## Creating network providers

It's recommended to use the `dharitri_py_sdk_network_providers` components **as a starting point**. As your application matures, switch to using your own network provider (e.g. deriving from the default ones), tailored to your requirements.

Creating an API provider:

In [ ]:
from dharitri_py_sdk import ApiNetworkProvider

provider = ApiNetworkProvider("https://devnet-api.dharitri.org")

Creating a Proxy provider:

In [ ]:
from dharitri_py_sdk import ProxyNetworkProvider

provider = ProxyNetworkProvider("https://devnet-gateway.dharitri.org")

## Fetching network parameters

In order to fetch network parameters, do as follows:

In [ ]:
config = provider.get_network_config()

print("Chain ID:", config.chain_id)
print("Min gas price:", config.min_gas_price)

## Fetching account state

The following snippet fetches (from the Network) the **nonce** and the **balance** of an account:

In [ ]:
account_on_network = provider.get_account(alice)

print("Nonce:", account_on_network.nonce)
print("Balance:", account_on_network.balance)

When sending a number of transactions, you usually have to first fetch the account nonce from the network (see above), then manage it locally (e.g. increment upon signing & broadcasting a transaction):

In [ ]:
from dharitri_py_sdk import AccountNonceHolder

nonce_holder = AccountNonceHolder(account_on_network.nonce)

tx.nonce = nonce_holder.get_nonce_then_increment()
# Then, sign & broadcast the transaction(s).

For further reference, please see [nonce management](https://docs.dharitri.org/integrators/creating-transactions/#nonce-management).

## Broadcasting transactions

Broadcast a single transaction:

In [ ]:
alice = Address.new_from_bech32("drt1c7pyyq2yaq5k7atn9z6qn5qkxwlc6zwc4vg7uuxn9ssy7evfh5jq4nm79l")

tx = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

alice_on_network = provider.get_account(alice)

tx.nonce = alice_on_network.nonce
tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx))

hash = provider.send_transaction(tx)
print("Transaction hash:", hash)

Broadcast multiple transactions:

In [ ]:
tx_1 = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

tx_2 = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

tx_3 = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

alice_on_network = provider.get_account(alice)
nonce_holder = AccountNonceHolder(account_on_network.nonce)

tx_1.nonce = nonce_holder.get_nonce_then_increment()
tx_2.nonce = nonce_holder.get_nonce_then_increment()
tx_3.nonce = nonce_holder.get_nonce_then_increment()

tx_1.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx_1))
tx_2.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx_2))
tx_3.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx_3))

hashes = provider.send_transactions([tx_1, tx_2, tx_3])
print("Transactions hashes:", hashes)

Now let's fetch a previously-broadcasted transaction:

In [ ]:
tx_on_network = provider.get_transaction("9270a6879b682a7b310c659f58b641ccdd5f083e5633669817130269e5b0939b", with_process_status=True)
print("Status:", tx_on_network.status)
print("Is completed:", tx_on_network.is_completed)